In [36]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
import pyspiel
import open_spiel.python.games
from open_spiel.python.observation import make_observation
from open_spiel.python.examples.ubc_decorators import TakeSingleActionDecorator
from open_spiel.python.examples.straightforward_agent import StraightforwardAgent
from open_spiel.python.pytorch.mutations import *
from open_spiel.python.examples.ppo_utils import EnvParams
from open_spiel.python.examples.ppo_eval import eval_agents_parallel
from open_spiel.python.env_decorator import *

from auctions.webutils import *
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

import sys
import glob
import pandas as pd
import os
import seaborn as sns

from tqdm import tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import pickle
import re
import json
import numpy as np 
import pandas as pd
from absl import logging
import tempfile
import subprocess

import open_spiel.python.games
from open_spiel.python.examples.ubc_utils import *
from open_spiel.python.algorithms.exploitability import nash_conv, best_response

import bokeh
from bokeh.io import curdoc
from bokeh.plotting import figure, show, output_file, save
from bokeh.layouts import row, column
from bokeh.io import output_notebook
from bokeh.models import HoverTool, ColumnDataSource, ColorBar, LogColorMapper, LinearColorMapper
from bokeh.transform import linear_cmap, log_cmap, factor_cmap
from bokeh.palettes import Category10_10, Magma256, Spectral10, Category20_20
from bokeh.models import LinearAxis, Range1d
from bokeh.models import ColumnDataSource, FactorRange

from bokeh.resources import CDN
from bokeh.embed import file_html

from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns
from django.db.models import F
from auctions.models import *

import bokeh
from bokeh.layouts import row, column
from bokeh.plotting import figure, show, output_file, save
from bokeh.io import output_notebook
from bokeh.models import HoverTool, ColumnDataSource, ColorBar, LogColorMapper, LinearColorMapper
from bokeh.transform import linear_cmap, log_cmap
from bokeh.palettes import Category10_10, Magma256, Spectral10, Category20_20
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from open_spiel.python.examples.ubc_cma import *

import itertools
output_notebook()
from open_spiel.python.examples.ubc_cma import *
from open_spiel.python.examples.env_and_policy import *
from open_spiel.python.examples.ppo_eval import run_eval
from open_spiel.python.examples.straightforward_agent import StraightforwardAgent
import copy

Loading BokehJS ...

In [38]:
PYSATS = '/apps/sats/python'

In [39]:
GAME_NAMES = [
    'regional_vs_national',
    'confused_national',
    'all_confused',
    'all_confused_with_local',
    '2rvn'
]


In [40]:
all_mods = defaultdict()

In [47]:
PREFIX = 'jan12'

In [50]:
# Generate all of the settings
for game_name in GAME_NAMES:
    with open(f'{PYSATS}/{game_name}.yml', 'rb') as f:
        settings = yaml.safe_load(f)

    mods = defaultdict()
    mods['base'] = settings

    # Mod 1: Change to hide demand
    hide_demand = copy.deepcopy(settings)
    hide_demand['auction_params']['information_policy'] = 'hide_demand'
    mods['hide_demand'] = hide_demand

    # TODO: This is broken right now b/c of national bidders cheesing it 
    # # Mod 2: Add a signal
    # signal = copy.deepcopy(settings)
    # signal['signal'] = 1
    # mods['signal'] = signal
    
    # Mod 3: Allow undersell
    undersell = copy.deepcopy(settings)
    undersell['auction_params']['undersell_rule'] = 'undersell_allowed'
    mods['undersell'] = undersell

    # # Mod 4: Decrease agent memory
    # memory = copy.deepcopy(settings)
    # memory['auction_params']['agent_memory'] = 1
    # mods['memory'] = memory
    
    # Mod 5: Reveal signal
    reveal_types = copy.deepcopy(settings)
    reveal_types['auction_params']['reveal_type_round'] = 2
    mods['reveal'] = reveal_types
    
    all_mods[game_name] = mods

In [51]:
# Write out YML files
OUTPUT_DIR = f'{PYSATS}/{PREFIX}'
!mkdir -p {OUTPUT_DIR}
for game_name in GAME_NAMES:
    for mod, mod_settings in all_mods[game_name].items():
        with open(f'{OUTPUT_DIR}/{game_name}_{mod}.yml', 'w') as f:
            yaml.dump(mod_settings, f)


In [ ]:
CONFIG_DIR = os.environ['CLOCK_AUCTION_CONFIG_DIR']

for game_name in GAME_NAMES:
    for mod, mod_settings in all_mods[game_name].items():
        !python {PYSATS}/pysats.py --config_file {OUTPUT_DIR}/{game_name}_{mod}.yml --seed 777 --output_file {CONFIG_DIR}/{PREFIX}_{game_name}_{mod}.json